In [3]:
import json
import pandas as pd
import numpy as np
import networkx as nx
import jellyfish
import os
import shutil
import subprocess
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt
from fuzzywuzzy import fuzz
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn import preprocessing
from sklearn.cluster import AgglomerativeClustering
from zipfile import ZipFile
from filecmp import dircmp

C:\ProgramData\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [57]:
n_clusters = [50,100,150,200,250,300]
affinity = ['euclidean', 'l1', 'l2', 'manhattan', 'cosine']
linkage = ['complete', 'average', 'single']


### Uncomment below for testing purposes ###
#n_clusters = [50]
#affinity = ['euclidean']
#linkage = ['complete']


def alvin_god(user_n_cluster, user_affinity, user_linkage, dir_arr, depends_arr, file_version, main_dict):
    vectorizer = TfidfVectorizer(stop_words='english')
    X = vectorizer.fit_transform(dir_arr)

    true_k = user_n_cluster
    model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
    model.fit(X)

    #print("Top terms per cluster:")
    order_centroids = model.cluster_centers_.argsort()[:, ::-1]
    terms = vectorizer.get_feature_names()
    
    
    
    
    
    
    main_cluster_arr = []
    col_counter = 0

    
    cluster = AgglomerativeClustering(n_clusters=user_n_cluster, affinity=user_affinity, linkage=user_linkage)
    cluster_result = cluster.fit_predict(depends_arr)

    cluster_result_arr = []
    cluster_counter = 0
    for element in cluster_result:
        #print(element)
        cluster_result_arr.append(element)

    main_cluster_arr.append(cluster_result_arr)
    
    col_counter += 1
    
    dir_arr_int = []
    #for element in dir_arr:
    #    dir_arr_int.append(main_dict[element])
    
    #cluster_result_int = []
    #for element in cluster_result:
        #cluster_result_int.append(main_dict[main_dict_val[element]])
    
    #print(dir_arr_int)
    #print('fk')
    #print(cluster_result_int)
    #common_results_rsf = common_member(dir_arr_int,cluster_result_int)
    #print(len(common_results_rsf))
    
    
    filename1 = 'MoJo_1.2.1/apache_spark/' +str(file_version) + '_' + str(user_n_cluster) + '_' + str(user_affinity) + "_" + str(user_linkage) + '_b' +".rsf"
    #print(filename1)
    with open(filename1, 'w') as f:
        for i in range(len(dir_arr)):
            Y = vectorizer.transform([dir_arr[i]])
            string = "contain " + str(model.predict(Y)[0]) + " " +str(i) + "\n"
            f.write(string)

        f.close()
    
    filename2 = 'MoJo_1.2.1/apache_spark/' +str(file_version) + '_' + str(user_n_cluster) + '_' + str(user_affinity) + "_" + str(user_linkage) + '_a' +".rsf"
    with open(filename2, 'w') as f:
        for i in range(len(cluster_result)):
            #print(element)
            string = "contain " + str(cluster_result[i]) + " " + str(i) + "\n"
            f.write(string)
        f.close()
        #for element in cluster_result:
            #print(element)
            #string = "contain " + str(element) + " " + str(main_dict[main_dict_val[element]]) + "\n"
            #f.write(string)
        #f.close()

    #print('Difference is: ' + str(len(dir_arr) - len(G.nodes) ))
    if len(dir_arr) - len(G.nodes) > 0:
        with open(filename2,'a') as f:
            for i in range(len(dir_arr)-len(G.nodes)):
                tbc = (i+len(G.nodes))
                string = "contain " + str(tbc) + " " + str(tbc) + "\n"
                f.write(string)

            f.close()
    else:
        with open(filename1,'a') as f:
            for i in range(len(G.nodes) - len(dir_arr)):
                tbc = i+len(dir_arr)
                string = "contain " + str(tbc) + " " + str(tbc) + "\n"
                f.write(string)
            f.close()

    return filename1, filename2




def fileIsSame(right, left, path):
    return os.path.exists (os.path.join(left, path.replace(right, '')));

def compare(right, left):
    difference = list();
    for root, dirs, files in os.walk(right):
        for name in files:
            path = os.path.join(root, name);
            # check if file is same
            if fileIsSame(right, left, path):
                if os.path.isdir(path):
                    # recursively check subdirs
                    difference.extend(compare(path, left));
            else:
                # count file as difference
                difference.append(path);

    return difference;

def compare_similar(right, left):
    difference = list();
    for root, dirs, files in os.walk(right):
        for name in files:
            path = os.path.join(root, name);
            # check if file is same
            if fileIsSame(right, left, path):
                
                if os.path.isdir(path):
                    # recursively check subdirs
                    difference.extend(compare_similar(path, left));
                difference.append(path)
                
            #else:
                # count file as difference
                #difference.append(path);

    return difference;



def common_member(a,b):
    a_set = set(a)
    b_set = set(b)
    if len(a_set.intersection(b_set)) > 0:
        return(a_set.intersection(b_set))
    else:
        return set()
    
    


In [ ]:
### Run only to unzip source code

rootdir = 'raw_sourcecode_zip'
dir_arr = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        print(os.path.join(subdir, file))
        name = str(os.path.join(subdir, file)).split('\\')
        name = name[1]
        with ZipFile(os.path.join(subdir, file), 'r') as zipObj:
            zipObj.extractall('raw_sourcecode/' + name)
        ### Only look for Java Files ###
        #if ".java" in str(os.path.join(subdir, file)):
            #print(os.path.join(subdir, file))
            #dir_arr.append(str(os.path.join(subdir, file)))
            
print(len(dir_arr))

In [ ]:
rootdir = 'raw_sourcecode'
dir_arr = []
depth=2

for root, dirs, files in os.walk(rootdir):
    if root.count(os.sep) == depth and "Apache_Spark_versions" in str(root):
        print(root)
        dir_arr.append(root)
        #for file in files: 
            #print(os.path.join(subdir, file))
        ### Only look for Java Files ###
        #if "Apache Spark versions" in str(os.path.join(subdir, file)):
            #print(os.path.join(subdir, file))
        
            #dir_arr.append(str(os.path.join(subdir, file)))
            
#print(len(dir_arr))

In [ ]:
import os

for i in range(len(dir_arr)):
    #print(element)
    command = 'cd C:/Users/tanji/Desktop/FIT4003/depends-0.9.2 & ' + 'java -jar depends.jar java C:/Users/tanji/Desktop/FIT4003/raw_sourcecode/Apache_Spark_versions/' + dir_arr[i].split('\\')[-1] +  ' ../raw_depends/apache_spark/' + str(i) + '_' + dir_arr[i].split('\\')[-1] 
    print(command)
    #os.system('cd C:/Users/tanji/Desktop/FIT4003/MoJo_1.2.1 & java MoJo test/test2_a.rsf test/test2_b.rsf > test/test_results.txt')
    os.system(command)

In [ ]:
pwd

In [7]:
rootdir = 'raw_depends/apache_spark'
json_name_dict = {}
json_result_dict = {}
for root, dirs, files in os.walk(rootdir):
   
    print(files)
    for element in files:
        
        with open(rootdir + '/' + element) as f:
            tmp = json.load(f)
        
        element = element.split('_')
        json_name_dict[element[0]] = element[1]
        
        
        
        json_result_dict[element[0]] = tmp
        

['0_spark-1.6.0.json', '10_spark-2.1.3.json', '11_spark-2.2.0.json', '12_spark-2.2.1.json', '13_spark-2.2.2.json', '14_spark-2.2.3.json', '15_spark-2.3.0.json', '16_spark-2.3.1.json', '17_spark-2.3.2.json', '18_spark-2.3.3.json', '19_spark-2.4.0.json', '1_spark-1.6.1.json', '20_spark-2.4.1.json', '21_spark-2.4.2.json', '22_spark-2.4.3.json', '23_spark-2.4.4.json', '24_test.json', '2_spark-1.6.2.json', '3_spark-1.6.3.json', '4_spark-2.0.0.json', '5_spark-2.0.1.json', '6_spark-2.0.2.json', '7_spark-2.1.0.json', '8_spark-2.1.1.json', '9_spark-2.1.2.json']


In [8]:
json_name_dict

{'0': 'spark-1.6.0.json',
 '10': 'spark-2.1.3.json',
 '11': 'spark-2.2.0.json',
 '12': 'spark-2.2.1.json',
 '13': 'spark-2.2.2.json',
 '14': 'spark-2.2.3.json',
 '15': 'spark-2.3.0.json',
 '16': 'spark-2.3.1.json',
 '17': 'spark-2.3.2.json',
 '18': 'spark-2.3.3.json',
 '19': 'spark-2.4.0.json',
 '1': 'spark-1.6.1.json',
 '20': 'spark-2.4.1.json',
 '21': 'spark-2.4.2.json',
 '22': 'spark-2.4.3.json',
 '23': 'spark-2.4.4.json',
 '24': 'test.json',
 '2': 'spark-1.6.2.json',
 '3': 'spark-1.6.3.json',
 '4': 'spark-2.0.0.json',
 '5': 'spark-2.0.1.json',
 '6': 'spark-2.0.2.json',
 '7': 'spark-2.1.0.json',
 '8': 'spark-2.1.1.json',
 '9': 'spark-2.1.2.json'}

In [9]:
### Getting the ground truth by comparing previous 10 versions (Burden AF)

ground_truth_dict = {}

root_dir = 'raw_sourcecode/Apache_Spark_versions/'
for i in range(10,24):
    file1 = str(root_dir +json_name_dict[str(i)]).replace('.json', '') + '/'
    for j in range(1,11):
        file2 = str(root_dir + json_name_dict[str(i-j)]).replace('.json', '') + '/'
        
        #test = compare_similar()
        print(file1, file2)
        if (j == 1):
            prev_tmp = compare_similar(file1, file2)
        else:
            tmp = compare_similar(file1, file2)
            tmp = common_member(prev_tmp, tmp)
            #print(len(tmp))
            prev_tmp = tmp
    print(len(prev_tmp))
    ground_truth_dict[str(i)] = prev_tmp
        
        

raw_sourcecode/Apache_Spark_versions/spark-2.1.3/ raw_sourcecode/Apache_Spark_versions/spark-2.1.2/
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/ raw_sourcecode/Apache_Spark_versions/spark-2.1.1/
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/ raw_sourcecode/Apache_Spark_versions/spark-2.1.0/
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/ raw_sourcecode/Apache_Spark_versions/spark-2.0.2/
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/ raw_sourcecode/Apache_Spark_versions/spark-2.0.1/
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/ raw_sourcecode/Apache_Spark_versions/spark-2.0.0/
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/ raw_sourcecode/Apache_Spark_versions/spark-1.6.3/
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/ raw_sourcecode/Apache_Spark_versions/spark-1.6.2/
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/ raw_sourcecode/Apache_Spark_versions/spark-1.6.1/
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/ raw_sourcecode/Apache_Spark_versions/spark-1.6.0/


raw_sourcecode/Apache_Spark_versions/spark-2.3.3/ raw_sourcecode/Apache_Spark_versions/spark-2.3.0/
raw_sourcecode/Apache_Spark_versions/spark-2.3.3/ raw_sourcecode/Apache_Spark_versions/spark-2.2.3/
raw_sourcecode/Apache_Spark_versions/spark-2.3.3/ raw_sourcecode/Apache_Spark_versions/spark-2.2.2/
raw_sourcecode/Apache_Spark_versions/spark-2.3.3/ raw_sourcecode/Apache_Spark_versions/spark-2.2.1/
raw_sourcecode/Apache_Spark_versions/spark-2.3.3/ raw_sourcecode/Apache_Spark_versions/spark-2.2.0/
raw_sourcecode/Apache_Spark_versions/spark-2.3.3/ raw_sourcecode/Apache_Spark_versions/spark-2.1.3/
raw_sourcecode/Apache_Spark_versions/spark-2.3.3/ raw_sourcecode/Apache_Spark_versions/spark-2.1.2/
raw_sourcecode/Apache_Spark_versions/spark-2.3.3/ raw_sourcecode/Apache_Spark_versions/spark-2.1.1/
13417
raw_sourcecode/Apache_Spark_versions/spark-2.4.0/ raw_sourcecode/Apache_Spark_versions/spark-2.3.3/
raw_sourcecode/Apache_Spark_versions/spark-2.4.0/ raw_sourcecode/Apache_Spark_versions/spark-2

In [10]:
#C:/Users/tanji/Desktop/FIT4003/raw_sourcecode/Apache_Spark_versions/spark-1.6.0', 'C:/Users/tanji/Desktop/FIT4003/raw_sourcecode/Apache_Spark_versions/spark-2.4.4') 
#print(compare('raw_sourcecode/Apache_Spark_versions/spark-1.6.0/', 'raw_sourcecode/Apache_Spark_versions/test/spark-2.4.4'));

In [11]:
test = compare_similar('raw_sourcecode/Apache_Spark_versions/spark-1.6.0/', 'raw_sourcecode/Apache_Spark_versions/spark-2.4.4/')
test2 = compare_similar('raw_sourcecode/Apache_Spark_versions/spark-1.6.0/', 'raw_sourcecode/Apache_Spark_versions/spark-2.2.0/')
test3 = compare_similar('raw_sourcecode/Apache_Spark_versions/spark-1.6.0/', 'raw_sourcecode/Apache_Spark_versions/spark-2.2.3/')
print(len(test))
test4 = common_member(test,test2)
test5 = common_member(test4,test3)
print(len(test5))

11769
11768


In [12]:
test[0]

'raw_sourcecode/Apache_Spark_versions/spark-1.6.0/.gitattributes'

In [13]:
print(len(ground_truth_dict['10']))
print(len(ground_truth_dict))

11962
14


In [14]:
for element in ground_truth_dict['10']:
    print(element)

raw_sourcecode/Apache_Spark_versions/spark-2.1.3/sql\hive\src\test\resources\golden\correlationoptimizer9-7-f952899d70bd718cbdbc44a5290938c9
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/sql\hive\src\test\resources\golden\union26-2-5c88f10d25ae39c1e2df8f4beaa8d562
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/sql\hive\src\test\resources\ql\src\test\queries\clientpositive\exim_22_import_exist_authsuccess.q
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/docs\img\ml-PipelineModel.png
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/python\test_support\sql\parquet_partitioned\year=2015\month=10\day=26\part-r-00005.gz.parquet
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/sql\hive\src\test\resources\ql\src\test\queries\clientpositive\authorization_parts.q
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/core\src\test\scala\org\apache\spark\SSLSampleConfigs.scala
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/sql\hive\src\test\resources\ql\src\test\queries\clientpositive\stats_co

raw_sourcecode/Apache_Spark_versions/spark-2.1.3/sql\hive\src\test\resources\golden\ppd_gby2-5-25541db999d8c1d56ba36b63949b6073
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/sql\catalyst\src\test\scala\org\apache\spark\sql\catalyst\expressions\aggregate\HyperLogLogPlusPlusSuite.scala
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/sbin\stop-slave.sh
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/sql\hive\src\test\resources\golden\stats_publisher_error_1-9-3b7c5bcb71c75e913caaea3acd48530c
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/sql\hive\src\test\resources\ql\src\test\queries\clientpositive\udf_acos.q
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/sql\hive\src\test\resources\golden\router_join_ppr-5-89b4fc284984f51c12f15017707ad312
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/sql\hive\src\test\resources\golden\SPARK-10484 Optimize the Cartesian (Cross) Join with broadcast based JOIN #1-0-abfc0b99ee357f71639f6162345fe8e
raw_sourcecode/Apache_Spark_versions/spark-2.1.3

raw_sourcecode/Apache_Spark_versions/spark-2.1.3/sql\hive\src\test\resources\golden\protectmode-13-70e6f4e73e37e6288fafdf6b8da5c62c
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/sql\hive\src\test\resources\golden\udf_xpath_int-3-a04ed6832ab542d6ee5903039511a826
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/dev\create-release\known_translations
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/sql\hive\src\test\resources\ql\src\test\queries\clientpositive\insert1_overwrite_partitions.q
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/sql\hive\src\test\resources\golden\union26-3-4702087f160230463e6114d6534d47e4
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/sql\hive\src\test\resources\golden\smb_mapjoin_6-25-e3552fabb4c9920d9bb964da5b907b46
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/sql\core\src\main\java\org\apache\spark\sql\api\java\UDF5.java
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/sql\hive\src\test\resources\golden\cp_mj_rc-3-b1746cc42d6bbd2b1d314a9e5b18eb60
raw_

raw_sourcecode/Apache_Spark_versions/spark-2.1.3/sql\hive\src\test\resources\golden\correlationoptimizer14-13-5e19a84c0c3ede17b8d9685a22f0a1e6
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/examples\src\main\python\mllib\recommendation_example.py
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/sql\hive\src\test\resources\ql\src\test\queries\clientpositive\partition_wise_fileformat3.q
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/sql\hive\src\test\resources\golden\smb_mapjoin_4-21-b4e83e2474975db8ed614e671af132f2
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/sql\hive\src\test\resources\golden\partition_date-6-416ab10ac818d432cd3a81870d6e5164
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/sql\hive\src\test\resources\golden\udf_java_method-3-c526dfd4d9eac718ced9afb3cf9a62fd
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/sql\hive\src\test\resources\golden\auto_sortmerge_join_11-13-398b81a1928284f29e832838ec3764fd
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/sql\hive\src\te

raw_sourcecode/Apache_Spark_versions/spark-2.1.3/R\pkg\R\schema.R
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/sql\hive\src\test\resources\golden\groupby_grouping_sets2-2-d1c300ea08361fb8237689c6cf8cc1b5
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/core\src\main\scala\org\apache\spark\api\java\JavaSparkStatusTracker.scala
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/sql\hive\src\test\resources\ql\src\test\queries\clientpositive\partition_schema1.q
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/sql\hive\src\test\resources\golden\join_nulls-15-e9f9be8be6e936fb832483dfa0be5126
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/sql\hive\src\test\resources\golden\decimal_1-2-ee665100ca5de3a006df43e97cfa707
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/sql\hive\src\test\resources\golden\load_dyn_part10-3-dc129f70e75cd575ce8c0de288884523
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/sql\hive\src\test\resources\golden\inputddl5-0-ebbf2aec5f76af7225c2efaf870b8ba7
raw_sourcecod

raw_sourcecode/Apache_Spark_versions/spark-2.1.3/sql\core\src\test\scala\org\apache\spark\sql\DataFrameNaFunctionsSuite.scala
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/python\test_support\sql\parquet_partitioned\year=2015\month=10\day=26\.part-r-00005.gz.parquet.crc
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/sql\hive\src\test\resources\golden\smb_mapjoin_13-10-b07488dd0aa9072057d8264271e93495
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/sql\hive\src\test\resources\golden\case statements WITHOUT key #2-0-e3a2b981ebff7e273537dd6c43ece0c0
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/sql\hive\src\test\resources\golden\date_udf-15-d031ee50c119d7c6acafd53543dbd0c4
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/dev\run-tests
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/sql\hive\src\test\resources\golden\load_dyn_part1-8-ea921e0af59a4940a11c94143b1c4b32
raw_sourcecode/Apache_Spark_versions/spark-2.1.3/sql\hive\src\test\resources\golden\varchar_join1-13-8318072ee410ca

In [59]:
for i in range(len(ground_truth_dict)):
    #print(i)
    dir_arr = []
    for element in ground_truth_dict[str(i+10)]:
        if ".java" in element:
            element = element.replace('\\', '/')
            element = element.replace('raw_sourcecode/Apache_Spark_versions/' + json_name_dict[str(i+10)].split('.json')[0] + '/', '' )
            element = element.replace('/', '\\')
            element = '\\' + element
            dir_arr.append(element)
            
    print('raw_depends/apache_spark/' + str(i+10)+ '_'+ json_name_dict[str(i+10)])
    with open('raw_depends/apache_spark/' + str(i+10)+ '_'+ json_name_dict[str(i+10)]) as f:
        spark_results = json.load(f)
        
    main_dict = {}
    main_dict_val = {}
    main_dict_counter = 0
    dict_array = []
    min_set = set()
    for element in spark_results['variables']:
        #print(element)

        ### Replace this with the initial JSON directory ###
        #print(json_name_dict[str(i+10)])
        element_tbc = element.replace('C:\\Users\\tanji\\Desktop\\FIT4003\\raw_sourcecode\\Apache_Spark_versions\\' +json_name_dict[str(i+10)].split('.json')[0] , '')
        #print(element)
        main_dict[element_tbc] = main_dict_counter
        main_dict_val[main_dict_counter] = element_tbc
        main_dict_counter += 1
 
    
    index  = 0
    var_array = []
    for value in spark_results['variables']:
        var_array.append([index, value])
        #print(index, value)
        index += 1
    
    var_df = pd.DataFrame(var_array)
    var_df.columns = ['index_val', 'name']
    
    feature_list = {}
    feature_index = 2
    for element in spark_results['cells']:
        #print(element)
        try:
            for a in element['values']:
                if a not in feature_list:
                    feature_list[a] = feature_index
                    feature_index += 1
                #print(a['Call'])


        except:
            pass
        
    feature_arr = []
    for element in spark_results['cells']:
        #print(array)
        array = [0] * (len(feature_list) + 2)
        values = dict(element['values'])
        #print(element)
        array[0] = element['src']
        array[1] = element['dest']
        for feature in feature_list:
            try:
                value = values[feature]
                array[feature_list[feature]] = value
            except:
                pass
        #print(array)
        feature_arr.append(array)
        
    feature_df = pd.DataFrame(feature_arr)
    col_names = ['src', 'dest']
    for element in feature_list:
        col_names.append(element)
    feature_df.columns = col_names
    
    feature_df['sum'] = feature_df.sum(axis=1) - feature_df['src'] - feature_df['dest']
    G = nx.Graph()
    for index, row in feature_df.iterrows():
        G.add_edge(row['src'], row['dest'], weight=row['sum'])
        
    adj_mat = nx.adjacency_matrix(G)
    adj_mat_df = pd.DataFrame(adj_mat.todense())
    np.fill_diagonal(adj_mat_df.values, adj_mat_df.values.max())
    x = adj_mat_df.values
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    adj_mat_df = pd.DataFrame(x_scaled)
    
    filename_arr = []
    for number in n_clusters:
        for affinity_type in affinity:
            for linkage_type in linkage :
                filename_1, filename_2 = alvin_god(number, affinity_type, linkage_type, dir_arr, adj_mat_df, json_name_dict[str(i+10)].split('.json')[0], main_dict)
                filename_1 = filename_1.replace('MoJo_1.2.1/', '')
                filename_2 = filename_2.replace('MoJo_1.2.1/', '')
                filename_arr.append((filename_1, filename_2))
                print(filename_1, filename_2)
    
    
    for i in range(len(filename_arr)):
    
        command = 'cd C:/Users/tanji/Desktop/FIT4003/MoJo_1.2.1 & ' + 'java MoJo ' + filename_arr[i][1] + ' ' + filename_arr[i][0] + ' >> apache_spark/apache_spark_results.txt'
        #print(command)
        #os.system('cd C:/Users/tanji/Desktop/FIT4003/MoJo_1.2.1 & java MoJo test/test2_a.rsf test/test2_b.rsf > test/test_results.txt')
        os.system(command)

raw_depends/apache_spark/10_spark-2.1.3.json
apache_spark/spark-2.1.3_50_euclidean_complete_b.rsf apache_spark/spark-2.1.3_50_euclidean_complete_a.rsf
apache_spark/spark-2.1.3_50_euclidean_average_b.rsf apache_spark/spark-2.1.3_50_euclidean_average_a.rsf
apache_spark/spark-2.1.3_50_euclidean_single_b.rsf apache_spark/spark-2.1.3_50_euclidean_single_a.rsf
apache_spark/spark-2.1.3_50_l1_complete_b.rsf apache_spark/spark-2.1.3_50_l1_complete_a.rsf
apache_spark/spark-2.1.3_50_l1_average_b.rsf apache_spark/spark-2.1.3_50_l1_average_a.rsf
apache_spark/spark-2.1.3_50_l1_single_b.rsf apache_spark/spark-2.1.3_50_l1_single_a.rsf
apache_spark/spark-2.1.3_50_l2_complete_b.rsf apache_spark/spark-2.1.3_50_l2_complete_a.rsf
apache_spark/spark-2.1.3_50_l2_average_b.rsf apache_spark/spark-2.1.3_50_l2_average_a.rsf
apache_spark/spark-2.1.3_50_l2_single_b.rsf apache_spark/spark-2.1.3_50_l2_single_a.rsf
apache_spark/spark-2.1.3_50_manhattan_complete_b.rsf apache_spark/spark-2.1.3_50_manhattan_complete_a.r

apache_spark/spark-2.1.3_300_l2_single_b.rsf apache_spark/spark-2.1.3_300_l2_single_a.rsf
apache_spark/spark-2.1.3_300_manhattan_complete_b.rsf apache_spark/spark-2.1.3_300_manhattan_complete_a.rsf
apache_spark/spark-2.1.3_300_manhattan_average_b.rsf apache_spark/spark-2.1.3_300_manhattan_average_a.rsf
apache_spark/spark-2.1.3_300_manhattan_single_b.rsf apache_spark/spark-2.1.3_300_manhattan_single_a.rsf
apache_spark/spark-2.1.3_300_cosine_complete_b.rsf apache_spark/spark-2.1.3_300_cosine_complete_a.rsf
apache_spark/spark-2.1.3_300_cosine_average_b.rsf apache_spark/spark-2.1.3_300_cosine_average_a.rsf
apache_spark/spark-2.1.3_300_cosine_single_b.rsf apache_spark/spark-2.1.3_300_cosine_single_a.rsf
raw_depends/apache_spark/11_spark-2.2.0.json
apache_spark/spark-2.2.0_50_euclidean_complete_b.rsf apache_spark/spark-2.2.0_50_euclidean_complete_a.rsf
apache_spark/spark-2.2.0_50_euclidean_average_b.rsf apache_spark/spark-2.2.0_50_euclidean_average_a.rsf
apache_spark/spark-2.2.0_50_euclidean

apache_spark/spark-2.2.0_300_euclidean_average_b.rsf apache_spark/spark-2.2.0_300_euclidean_average_a.rsf
apache_spark/spark-2.2.0_300_euclidean_single_b.rsf apache_spark/spark-2.2.0_300_euclidean_single_a.rsf
apache_spark/spark-2.2.0_300_l1_complete_b.rsf apache_spark/spark-2.2.0_300_l1_complete_a.rsf
apache_spark/spark-2.2.0_300_l1_average_b.rsf apache_spark/spark-2.2.0_300_l1_average_a.rsf
apache_spark/spark-2.2.0_300_l1_single_b.rsf apache_spark/spark-2.2.0_300_l1_single_a.rsf
apache_spark/spark-2.2.0_300_l2_complete_b.rsf apache_spark/spark-2.2.0_300_l2_complete_a.rsf
apache_spark/spark-2.2.0_300_l2_average_b.rsf apache_spark/spark-2.2.0_300_l2_average_a.rsf
apache_spark/spark-2.2.0_300_l2_single_b.rsf apache_spark/spark-2.2.0_300_l2_single_a.rsf
apache_spark/spark-2.2.0_300_manhattan_complete_b.rsf apache_spark/spark-2.2.0_300_manhattan_complete_a.rsf
apache_spark/spark-2.2.0_300_manhattan_average_b.rsf apache_spark/spark-2.2.0_300_manhattan_average_a.rsf
apache_spark/spark-2.2.0

apache_spark/spark-2.2.1_250_manhattan_complete_b.rsf apache_spark/spark-2.2.1_250_manhattan_complete_a.rsf
apache_spark/spark-2.2.1_250_manhattan_average_b.rsf apache_spark/spark-2.2.1_250_manhattan_average_a.rsf
apache_spark/spark-2.2.1_250_manhattan_single_b.rsf apache_spark/spark-2.2.1_250_manhattan_single_a.rsf
apache_spark/spark-2.2.1_250_cosine_complete_b.rsf apache_spark/spark-2.2.1_250_cosine_complete_a.rsf
apache_spark/spark-2.2.1_250_cosine_average_b.rsf apache_spark/spark-2.2.1_250_cosine_average_a.rsf
apache_spark/spark-2.2.1_250_cosine_single_b.rsf apache_spark/spark-2.2.1_250_cosine_single_a.rsf
apache_spark/spark-2.2.1_300_euclidean_complete_b.rsf apache_spark/spark-2.2.1_300_euclidean_complete_a.rsf
apache_spark/spark-2.2.1_300_euclidean_average_b.rsf apache_spark/spark-2.2.1_300_euclidean_average_a.rsf
apache_spark/spark-2.2.1_300_euclidean_single_b.rsf apache_spark/spark-2.2.1_300_euclidean_single_a.rsf
apache_spark/spark-2.2.1_300_l1_complete_b.rsf apache_spark/spar

apache_spark/spark-2.2.2_250_euclidean_single_b.rsf apache_spark/spark-2.2.2_250_euclidean_single_a.rsf
apache_spark/spark-2.2.2_250_l1_complete_b.rsf apache_spark/spark-2.2.2_250_l1_complete_a.rsf
apache_spark/spark-2.2.2_250_l1_average_b.rsf apache_spark/spark-2.2.2_250_l1_average_a.rsf
apache_spark/spark-2.2.2_250_l1_single_b.rsf apache_spark/spark-2.2.2_250_l1_single_a.rsf
apache_spark/spark-2.2.2_250_l2_complete_b.rsf apache_spark/spark-2.2.2_250_l2_complete_a.rsf
apache_spark/spark-2.2.2_250_l2_average_b.rsf apache_spark/spark-2.2.2_250_l2_average_a.rsf
apache_spark/spark-2.2.2_250_l2_single_b.rsf apache_spark/spark-2.2.2_250_l2_single_a.rsf
apache_spark/spark-2.2.2_250_manhattan_complete_b.rsf apache_spark/spark-2.2.2_250_manhattan_complete_a.rsf
apache_spark/spark-2.2.2_250_manhattan_average_b.rsf apache_spark/spark-2.2.2_250_manhattan_average_a.rsf
apache_spark/spark-2.2.2_250_manhattan_single_b.rsf apache_spark/spark-2.2.2_250_manhattan_single_a.rsf
apache_spark/spark-2.2.2_2

...

apache_spark/spark-2.4.1_50_cosine_average_b.rsf apache_spark/spark-2.4.1_50_cosine_average_a.rsf
apache_spark/spark-2.4.1_50_cosine_single_b.rsf apache_spark/spark-2.4.1_50_cosine_single_a.rsf
apache_spark/spark-2.4.1_100_euclidean_complete_b.rsf apache_spark/spark-2.4.1_100_euclidean_complete_a.rsf
apache_spark/spark-2.4.1_100_euclidean_average_b.rsf apache_spark/spark-2.4.1_100_euclidean_average_a.rsf
apache_spark/spark-2.4.1_100_euclidean_single_b.rsf apache_spark/spark-2.4.1_100_euclidean_single_a.rsf
apache_spark/spark-2.4.1_100_l1_complete_b.rsf apache_spark/spark-2.4.1_100_l1_complete_a.rsf
apache_spark/spark-2.4.1_100_l1_average_b.rsf apache_spark/spark-2.4.1_100_l1_average_a.rsf
apache_spark/spark-2.4.1_100_l1_single_b.rsf apache_spark/spark-2.4.1_100_l1_single_a.rsf
apache_spark/spark-2.4.1_100_l2_complete_b.rsf apache_spark/spark-2.4.1_100_l2_complete_a.rsf
apache_spark/spark-2.4.1_100_l2_average_b.rsf apache_spark/spark-2.4.1_100_l2_average_a.rsf
apache_spark/spark-2.4.1_1

apache_spark/spark-2.4.2_50_l2_complete_b.rsf apache_spark/spark-2.4.2_50_l2_complete_a.rsf
apache_spark/spark-2.4.2_50_l2_average_b.rsf apache_spark/spark-2.4.2_50_l2_average_a.rsf
apache_spark/spark-2.4.2_50_l2_single_b.rsf apache_spark/spark-2.4.2_50_l2_single_a.rsf
apache_spark/spark-2.4.2_50_manhattan_complete_b.rsf apache_spark/spark-2.4.2_50_manhattan_complete_a.rsf
apache_spark/spark-2.4.2_50_manhattan_average_b.rsf apache_spark/spark-2.4.2_50_manhattan_average_a.rsf
apache_spark/spark-2.4.2_50_manhattan_single_b.rsf apache_spark/spark-2.4.2_50_manhattan_single_a.rsf
apache_spark/spark-2.4.2_50_cosine_complete_b.rsf apache_spark/spark-2.4.2_50_cosine_complete_a.rsf
apache_spark/spark-2.4.2_50_cosine_average_b.rsf apache_spark/spark-2.4.2_50_cosine_average_a.rsf
apache_spark/spark-2.4.2_50_cosine_single_b.rsf apache_spark/spark-2.4.2_50_cosine_single_a.rsf
apache_spark/spark-2.4.2_100_euclidean_complete_b.rsf apache_spark/spark-2.4.2_100_euclidean_complete_a.rsf
apache_spark/spa

apache_spark/spark-2.4.2_300_cosine_single_b.rsf apache_spark/spark-2.4.2_300_cosine_single_a.rsf
raw_depends/apache_spark/22_spark-2.4.3.json
apache_spark/spark-2.4.3_50_euclidean_complete_b.rsf apache_spark/spark-2.4.3_50_euclidean_complete_a.rsf
apache_spark/spark-2.4.3_50_euclidean_average_b.rsf apache_spark/spark-2.4.3_50_euclidean_average_a.rsf
apache_spark/spark-2.4.3_50_euclidean_single_b.rsf apache_spark/spark-2.4.3_50_euclidean_single_a.rsf
apache_spark/spark-2.4.3_50_l1_complete_b.rsf apache_spark/spark-2.4.3_50_l1_complete_a.rsf
apache_spark/spark-2.4.3_50_l1_average_b.rsf apache_spark/spark-2.4.3_50_l1_average_a.rsf
apache_spark/spark-2.4.3_50_l1_single_b.rsf apache_spark/spark-2.4.3_50_l1_single_a.rsf
apache_spark/spark-2.4.3_50_l2_complete_b.rsf apache_spark/spark-2.4.3_50_l2_complete_a.rsf
apache_spark/spark-2.4.3_50_l2_average_b.rsf apache_spark/spark-2.4.3_50_l2_average_a.rsf
apache_spark/spark-2.4.3_50_l2_single_b.rsf apache_spark/spark-2.4.3_50_l2_single_a.rsf
apach

apache_spark/spark-2.4.3_300_l2_average_b.rsf apache_spark/spark-2.4.3_300_l2_average_a.rsf
apache_spark/spark-2.4.3_300_l2_single_b.rsf apache_spark/spark-2.4.3_300_l2_single_a.rsf
apache_spark/spark-2.4.3_300_manhattan_complete_b.rsf apache_spark/spark-2.4.3_300_manhattan_complete_a.rsf
apache_spark/spark-2.4.3_300_manhattan_average_b.rsf apache_spark/spark-2.4.3_300_manhattan_average_a.rsf
apache_spark/spark-2.4.3_300_manhattan_single_b.rsf apache_spark/spark-2.4.3_300_manhattan_single_a.rsf
apache_spark/spark-2.4.3_300_cosine_complete_b.rsf apache_spark/spark-2.4.3_300_cosine_complete_a.rsf
apache_spark/spark-2.4.3_300_cosine_average_b.rsf apache_spark/spark-2.4.3_300_cosine_average_a.rsf
apache_spark/spark-2.4.3_300_cosine_single_b.rsf apache_spark/spark-2.4.3_300_cosine_single_a.rsf
raw_depends/apache_spark/23_spark-2.4.4.json
apache_spark/spark-2.4.4_50_euclidean_complete_b.rsf apache_spark/spark-2.4.4_50_euclidean_complete_a.rsf
apache_spark/spark-2.4.4_50_euclidean_average_b.r

apache_spark/spark-2.4.4_300_euclidean_complete_b.rsf apache_spark/spark-2.4.4_300_euclidean_complete_a.rsf
apache_spark/spark-2.4.4_300_euclidean_average_b.rsf apache_spark/spark-2.4.4_300_euclidean_average_a.rsf
apache_spark/spark-2.4.4_300_euclidean_single_b.rsf apache_spark/spark-2.4.4_300_euclidean_single_a.rsf
apache_spark/spark-2.4.4_300_l1_complete_b.rsf apache_spark/spark-2.4.4_300_l1_complete_a.rsf
apache_spark/spark-2.4.4_300_l1_average_b.rsf apache_spark/spark-2.4.4_300_l1_average_a.rsf
apache_spark/spark-2.4.4_300_l1_single_b.rsf apache_spark/spark-2.4.4_300_l1_single_a.rsf
apache_spark/spark-2.4.4_300_l2_complete_b.rsf apache_spark/spark-2.4.4_300_l2_complete_a.rsf
apache_spark/spark-2.4.4_300_l2_average_b.rsf apache_spark/spark-2.4.4_300_l2_average_a.rsf
apache_spark/spark-2.4.4_300_l2_single_b.rsf apache_spark/spark-2.4.4_300_l2_single_a.rsf
apache_spark/spark-2.4.4_300_manhattan_complete_b.rsf apache_spark/spark-2.4.4_300_manhattan_complete_a.rsf
apache_spark/spark-2.4

In [29]:
dir_arr

['\\core\\src\\main\\java\\org\\apache\\spark\\util\\collection\\unsafe\\sort\\UnsafeSorterSpillWriter.java',
 '\\examples\\src\\main\\java\\org\\apache\\spark\\examples\\ml\\JavaDecisionTreeRegressionExample.java',
 '\\launcher\\src\\main\\java\\org\\apache\\spark\\launcher\\CommandBuilderUtils.java',
 '\\examples\\src\\main\\java\\org\\apache\\spark\\examples\\ml\\JavaNGramExample.java',
 '\\launcher\\src\\main\\java\\org\\apache\\spark\\launcher\\ChildProcAppHandle.java',
 '\\sql\\catalyst\\src\\main\\java\\org\\apache\\spark\\sql\\catalyst\\expressions\\codegen\\UnsafeArrayWriter.java',
 '\\graphx\\src\\main\\scala\\org\\apache\\spark\\graphx\\util\\package-info.java',
 '\\core\\src\\main\\java\\org\\apache\\spark\\shuffle\\sort\\UnsafeShuffleWriter.java',
 '\\sql\\hive\\src\\test\\java\\org\\apache\\spark\\sql\\hive\\execution\\UDFIntegerToString.java',
 '\\streaming\\src\\test\\java\\org\\apache\\spark\\streaming\\JavaAPISuite.java',
 '\\mllib\\src\\test\\java\\org\\apache\\spark

In [30]:
main_dict

{'\\common\\network-common\\src\\main\\java\\org\\apache\\spark\\network\\TransportContext.java': 0,
 '\\common\\network-common\\src\\main\\java\\org\\apache\\spark\\network\\buffer\\FileSegmentManagedBuffer.java': 1,
 '\\common\\network-common\\src\\main\\java\\org\\apache\\spark\\network\\buffer\\ManagedBuffer.java': 2,
 '\\common\\network-common\\src\\main\\java\\org\\apache\\spark\\network\\buffer\\NettyManagedBuffer.java': 3,
 '\\common\\network-common\\src\\main\\java\\org\\apache\\spark\\network\\buffer\\NioManagedBuffer.java': 4,
 '\\common\\network-common\\src\\main\\java\\org\\apache\\spark\\network\\client\\ChunkFetchFailureException.java': 5,
 '\\common\\network-common\\src\\main\\java\\org\\apache\\spark\\network\\client\\ChunkReceivedCallback.java': 6,
 '\\common\\network-common\\src\\main\\java\\org\\apache\\spark\\network\\client\\RpcResponseCallback.java': 7,
 '\\common\\network-common\\src\\main\\java\\org\\apache\\spark\\network\\client\\StreamCallback.java': 8,
 '\\

In [31]:
main_dict_val

{0: '\\common\\network-common\\src\\main\\java\\org\\apache\\spark\\network\\TransportContext.java',
 1: '\\common\\network-common\\src\\main\\java\\org\\apache\\spark\\network\\buffer\\FileSegmentManagedBuffer.java',
 2: '\\common\\network-common\\src\\main\\java\\org\\apache\\spark\\network\\buffer\\ManagedBuffer.java',
 3: '\\common\\network-common\\src\\main\\java\\org\\apache\\spark\\network\\buffer\\NettyManagedBuffer.java',
 4: '\\common\\network-common\\src\\main\\java\\org\\apache\\spark\\network\\buffer\\NioManagedBuffer.java',
 5: '\\common\\network-common\\src\\main\\java\\org\\apache\\spark\\network\\client\\ChunkFetchFailureException.java',
 6: '\\common\\network-common\\src\\main\\java\\org\\apache\\spark\\network\\client\\ChunkReceivedCallback.java',
 7: '\\common\\network-common\\src\\main\\java\\org\\apache\\spark\\network\\client\\RpcResponseCallback.java',
 8: '\\common\\network-common\\src\\main\\java\\org\\apache\\spark\\network\\client\\StreamCallback.java',
 9: 

In [ ]:
filename_arr

In [ ]:
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(dir_arr)

true_k = 300
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

#print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()

In [ ]:
Y = vectorizer.transform(['\\mllib\\src\\test\\java\\org\\apache\\spark\\ml\\classification\\JavaNaiveBayesSuite.java'])
prediction = model.predict(Y)
print(prediction)

Y = vectorizer.transform(['\\mllib\\src\\test\\java\\org\\apache\\spark\\mllib\\classification\\JavaNaiveBayesSuite.java'])
prediction = model.predict(Y)
print(prediction)

In [37]:
dir_arr_int

NameError: name 'dir_arr_int' is not defined